# Морфологічний аналіз

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
# Jupyter rendering stuff
from IPython.display import display, Math, Latex, HTML, clear_output

In [2]:
import itertools

## Input data

In [3]:
# def generate_probabilities(*sizes):
#     result = [np.random.random(size) for size in sizes]
#     for vec in result:
#         vec /= np.sum(vec)
#     return result

In [4]:
# def generate_cross_relation(size1, size2):
#     return np.random.random(size=(size1, size2)) * 2 - 1

In [5]:
# outer_f_s = (10, 10, 10, 10, 10, 10)
# outer_f_p = generate_probabilities(*outer_f_s)
# inner_f_s = (10, 10)
# inner_f_p = generate_probabilities(*inner_f_s)

In [6]:
# outer_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.combinations(enumerate(outer_f_s), 2)}
# inner_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.combinations(enumerate(inner_f_s), 2)}
# cross_rel = {(i, j):generate_cross_relation(s1, s2) for (i,s1), (j,s2) in itertools.product(enumerate(outer_f_s), enumerate(inner_f_s))}

In [7]:
outer_f_s = (2, 4, 4, 8, 3, 3, 7, 3, 3, 9) # внешние факторы - размеры столбиков
outer_f_p = [
    np.array([0.35,0.65]), # вероятности
    np.array([0.05, 0.30, 0.43, 0.22]),
    np.array([0.29, 0.21, 0.44, 0.06]),
    np.array([0.17, 0.09, 0.06, 0.12, 0.18, 0.17, 0.14, 0.06]),
    np.array([0.42, 0.18, 0.40]),
    np.array([0.51, 0.42, 0.07]),
    np.array([0.25, 0.20, 0.09, 0.04, 0.15, 0.11, 0.16]),
    np.array([0.53, 0.33, 0.13]),
    np.array([0.23, 0.34, 0.43]),
    np.array([0.19, 0.07, 0.02, 0.12, 0.10, 0.15, 0.19, 0.06, 0.09])
]

In [8]:
outer_rel = {(i, j):np.zeros((s1, s2)) for (i,s1), (j,s2) in itertools.combinations(enumerate(outer_f_s), 2)} # связи между внешними
# F1-F2
outer_rel[(0,1)][0] = [0.4, -0.1, 0.65, 0]
outer_rel[(0,1)][1] = [0, 0, 0, -0.7]
# F1-F3
outer_rel[(0,2)][0] = [0.4, 0,0,0]
outer_rel[(0,2)][1] = [0, 0, 0.43, 0]
# F1-F4
outer_rel[(0,3)][0] = [0.49, -0.20, -0.60, 0.10, 0.15, 0, 0, 0]
outer_rel[(0,3)][1] = [0.3, 0.08, 0, 0.3, 0.35, 0, 0, -0.9]
# F1-F9
outer_rel[(0,8)][1, 0] = 0.31

# F2-F3
outer_rel[(1,2)][0] = [0.47,0,0,0]
outer_rel[(1,2)][1] = [0, 0.28, 0.13, 0.09]
outer_rel[(1,2)][2] = [0.39,0,0.48,0]
outer_rel[(1,2)][3] = [0.41, 0, 0, 0]
# F2-F4
outer_rel[(1,3)][3, 2] = -0.47
outer_rel[(1,3)][0, 4] = 0.45
outer_rel[(1,3)][1, 7] = -0.12
# F2-F7
outer_rel[(1,6)][0,0] = -0.09
outer_rel[(1,6)][1,5] = 0.32
# F2-F10
outer_rel[(1,9)][1, 3] = 0.12

# F3-F4
outer_rel[(2,3)][1, 3] = -0.05
outer_rel[(2,3)][2, 2] = -0.20
outer_rel[(2,3)][3, 0] = -0.78
outer_rel[(2,3)][3, 2] = 0.87
outer_rel[(2,3)][3, 7] = -0.69
# F3-F7
outer_rel[(2,6)][0, 2] = -0.09
outer_rel[(2,6)][3, 5] = 0.23
# F3-F10
outer_rel[(2,9)][3, 3] = 0.4

# F4-F5
outer_rel[(3,4)][0, 0] = 0.80
outer_rel[(3,4)][6, 0] = 0.70
outer_rel[(3,4)][6, 1] = -0.09
outer_rel[(3,4)][7, 0] = -1.00
outer_rel[(3,4)][7, 1] = -1.00
# F4-F6
outer_rel[(3,5)][5, 2] = 0.15
# F4-F7
outer_rel[(3,6)][2, 0] = -0.07
# F4-F9
outer_rel[(3,8)][5, 2] = 0.30
outer_rel[(3,8)][7, 1] = 0.08
# F4-F10
outer_rel[(3,9)][1, 4] = 0.1

# F5-F6
outer_rel[(4,5)][2, 2] = 0.3
# F5-F7
outer_rel[(4,6)][0, 3] = 0.16
# F5-F8
outer_rel[(4,7)][1, 2] = -0.6
outer_rel[(4,7)][2, 2] = 0.9
# F5-F10
outer_rel[(4,9)][2, 4] = 0.43

# F6-F10
outer_rel[(5,9)][1, 1] = -0.04

# F7-F9
outer_rel[(6,8)][3, 2] = 0.50
# F7-F10
outer_rel[(6,9)][0, 2] = 0.27
outer_rel[(6,9)][6, 7] = 0.36

# F8-F9
outer_rel[(7,8)][2, 1] = -0.95
outer_rel[(7,8)][2, 2] = -0.44
# F8-F10
outer_rel[(7,9)][2, 0] = -0.83
outer_rel[(7,9)][2, 5] = -1

# F9-F10
outer_rel[(8,9)][0] = [0.5,0.4,0.1,0,0,0.45,0,0.3,0.2]
outer_rel[(8,9)][1] = [0.1,0,0,0.1,0,0,0.3,-0.7,-0.9]
outer_rel[(8,9)][2] = [0,0,0,0,0.15,0,0,0,0]

In [9]:
inner_f_s = (4, 4) 
inner_f_p = [
    np.array([0.4, 0.05, 0.4, 0.15]),
    np.array([0.5, 0.2, 0.25, 0.05])
]

In [10]:
cross_rel = {(0, 0): np.array([[-0.  , -0.46, -0.25,  0.22],
        [-0.61, -0.16, -0.19,  0.24]]),
 (0, 1): np.array([[ 0.11, -0.01,  0.46, -0.32],
        [-0.07, -0.44,  0.46,  0.42]]),
 (1, 0): np.array([[ 0.42, -0.19, -0.15, -0.12],
        [-0.05,  0.11,  0.21, -0.21],
        [-0.16,  0.08, -0.33, -0.41],
        [ 0.25, -0.05, -0.32, -0.15]]),
 (1, 1): np.array([[ 0.34,  0.07, -0.41, -0.03],
        [-0.18,  0.08, -0.13,  0.25],
        [ 0.15,  0.08, -0.36, -0.02],
        [ 0.28,  0.03, -0.29, -0.15]]),
 (2, 0): np.array([[ 0.11, -0.48, -0.16, -0.18],
        [ 0.06, -0.45,  0.  ,  0.01],
        [ 0.05, -0.26, -0.02, -0.13],
        [ 0.19, -0.39, -0.1 , -0.03]]),
 (2, 1): np.array([[-0.21,  0.03, -0.05, -0.24],
        [-0.28,  0.53, -0.1 ,  0.14],
        [ 0.24,  0.29,  0.13,  0.01],
        [ 0.27, -0.08,  0.24,  0.48]]),
 (3, 0): np.array([[ 0.21,  0.11,  0.1 , -0.02],
        [ 0.17,  0.45, -0.14,  0.58],
        [ 0.21,  0.14, -0.27,  0.62],
        [-0.08, -0.02,  0.02,  0.  ],
        [ 0.32, -0.48,  0.29, -0.25],
        [-0.05,  0.04,  0.3 , -0.13],
        [ 0.14, -0.02, -0.24, -0.14],
        [-0.5 , -0.24, -0.34,  0.42]]),
 (3, 1): np.array([[-0.29,  0.14, -0.29,  0.11],
        [-0.28,  0.06,  0.07, -0.39],
        [-0.2 ,  0.23,  0.28, -0.1 ],
        [ 0.15,  0.04,  0.18, -0.26],
        [ 0.3 ,  0.18, -0.25, -0.1 ],
        [ 0.31, -0.05,  0.13, -0.03],
        [ 0.02, -0.26, -0.13, -0.3 ],
        [-0.38,  0.05, -0.27, -0.27]]),
 (4, 0): np.array([[-0.05, -0.01, -0.15, -0.26],
        [-0.09, -0.07, -0.26, -0.03],
        [ 0.06,  0.03,  0.31,  0.01]]),
 (4, 1): np.array([[-0.16,  0.3 , -0.11, -0.11],
        [-0.43, -0.35, -0.2 , -0.68],
        [ 0.04,  0.1 ,  0.59,  0.16]]),
 (5, 0): np.array([[ 0.14, -0.03,  0.48, -0.43],
        [ 0.42,  0.16, -0.29, -0.05],
        [ 0.21, -0.03, -0.2 , -0.03]]),
 (5, 1): np.array([[-0.08,  0.01,  0.17, -0.24],
        [ 0.18,  0.16,  0.17, -0.54],
        [-0.19, -0.16, -0.26,  0.34]]),
 (6, 0): np.array([[-0.22,  0.24, -0.16,  0.1 ],
        [ 0.44,  0.22, -0.48,  0.12],
        [-0.21,  0.15, -0.35, -0.12],
        [ 0.05, -0.18,  0.02,  0.4 ],
        [-0.19,  0.08, -0.17,  0.12],
        [ 0.06, -0.13,  0.51,  0.05],
        [ 0.08, -0.16,  0.4 ,  0.16]]),
 (6, 1): np.array([[ 0.29,  0.13, -0.2 , -0.09],
        [-0.01,  0.17, -0.06,  0.22],
        [ 0.31, -0.18, -0.01,  0.03],
        [-0.24, -0.16,  0.27,  0.08],
        [ 0.04,  0.33,  0.14, -0.15],
        [-0.22, -0.18, -0.07, -0.01],
        [ 0.33,  0.09,  0.12,  0.36]]),
 (7, 0): np.array([[ 0.14, -0.11, -0.13,  0.1 ],
        [ 0.07,  0.09,  0.05, -0.04],
        [ 0.01,  0.12, -0.36,  0.03]]),
 (7, 1): np.array([[-0.3 ,  0.21,  0.18,  0.51],
        [-0.1 ,  0.41,  0.29,  0.07],
        [-0.4 , -0.04,  0.46, -0.11]]),
 (8, 0): np.array([[ 0.17, -0.1 ,  0.22,  0.13],
        [-0.06,  0.3 , -0.01,  0.4 ],
        [-0.43,  0.14,  0.2 ,  0.29]]),
 (8, 1): np.array([[ 0.15,  0.12, -0.59,  0.07],
        [-0.09,  0.2 ,  0.22, -0.03],
        [-0.15,  0.39,  0.03, -0.12]]),
 (9, 0): np.array([[-0.01, -0.23,  0.23,  0.56],
        [ 0.28,  0.12,  0.01,  0.17],
        [-0.14,  0.17,  0.15, -0.35],
        [ 0.22, -0.01, -0.02,  0.21],
        [ 0.24,  0.21,  0.  , -0.32],
        [ 0.46,  0.28,  0.19, -0.11],
        [-0.3 ,  0.08,  0.26, -0.28],
        [ 0.05,  0.22, -0.05, -0.39],
        [-0.39, -0.12,  0.02,  0.04]]),
 (9, 1): np.array([[-0.31, -0.53,  0.12,  0.38],
        [ 0.19, -0.12, -0.07,  0.13],
        [ 0.01, -0.28,  0.25,  0.12],
        [-0.97, -0.16, -0.37, -0.01],
        [ 0.17,  0.11,  0.26,  0.07],
        [ 0.16, -0.01, -0.14, -0.22],
        [ 0.4 ,  0.03, -0.03, -0.45],
        [ 0.24, -0.09,  0.03,  0.04],
        [ 0.28, -0.11, -0.27,  0.02]])}

## Calculating table for outer factors

In [11]:
df_outer = pd.DataFrame(index=pd.MultiIndex.from_product(tuple(range(size) for size in outer_f_s),
                                                         names=[str(i + 1) for i in range(len(outer_f_s))]))

In [12]:
def calculate_correlation(df, relation):
    factors = (label for label in df.index.labels)
    result = 1
    for (f1,a1), (f2, a2) in itertools.combinations(enumerate(factors), 2):
        result *= relation[f1, f2][a1, a2] + 1
    df['C'] = result

In [13]:
def calculate_cond_probs(df, factor_p):
    factors = (label for label in df.index.labels)
    probs = np.array([factor_p[i][factor] for i, factor in enumerate(factors)])
    probs = np.prod(probs, axis = 0) / probs
    for i in range(len(probs)):
        df_outer['P%d_' % (i + 1)] = probs[i] * df['C']

In [14]:
def normalize_cond_probs(df, factor_s):
    sn = [slice(None)] * len(factor_s)
    for i in range(len(factor_s)):
        sn_cur = sn.copy()
        df['P%d' % (i + 1)] = np.zeros(df.shape[0])
        for j in range(outer_f_s[i]):
            sn_cur[i] = j
            df.loc[tuple(sn_cur), 'P%d' % (i + 1)] = \
            df.loc[tuple(sn_cur), 'P%d_' % (i + 1)].as_matrix() / df.loc[tuple(sn_cur), 'P%d_' % (i + 1)].sum()

In [15]:
def generate_prob_matrices(df, factor_s):
    sn = [slice(None)] * len(factor_s)
    p_matrices = []
    for left in range(len(factor_s)):
        right = (left + 1) % len(factor_s)
        p = np.zeros((factor_s[left], factor_s[right]))
        for i, j in itertools.product(range(factor_s[left]), range(factor_s[right])):
            sn_cur = sn.copy()
            sn_cur[left], sn_cur[right] = i, j
            p[i,j] = df.loc[tuple(sn_cur),'P%d' % (right + 1)].sum()
        p_matrices.append(p)
    return p_matrices

In [16]:
def create_matrix_set(df, factor_s, factor_p, cross_rel):
    calculate_correlation(df, cross_rel)
    calculate_cond_probs(df, factor_p)
    normalize_cond_probs(df, factor_s)
    return generate_prob_matrices(df, factor_s)

In [17]:
def generate_cycle_product(matrices):
    result = [mat.copy() for mat in matrices]
    cycle_iter = itertools.cycle(matrices)
    for i in range(len(matrices) - 1):
        next(cycle_iter)
        for i in range(len(matrices)):
            result[i] = result[i] @ next(cycle_iter)
    return result

In [18]:
def find_eigenvec_near_one(matrices, eps=1e-8):
    result = []
    for matrix in matrices:
        vals, vecs = np.linalg.eig(matrix)
        mask = np.isclose(vals, 1, atol=eps)
        needed_vec = vecs[:, mask].flatten()
        needed_vec = np.real_if_close(needed_vec)
        result.append(needed_vec / needed_vec.sum())
        if (result[-1] < 0).any():
            raise RuntimeWarning("Probability vector has negative elements.")
    return result

In [19]:
outer_matrices = create_matrix_set(df_outer, outer_f_s, outer_f_p, outer_rel)
outer_matrices_cycled = generate_cycle_product(outer_matrices)
outer_real_probs = find_eigenvec_near_one(outer_matrices_cycled)

## Calculating effectiveness R

In [21]:
def calculate_cross_correlation(df, cross_rel, other_f_p):
    for other_f in range(len(other_f_p)):
        result = 1
        for f1,a1 in enumerate(label for label in df.index.labels):
            result *= cross_rel[f1, other_f][a1, :] + 1
        result = result * other_f_p[other_f] # p' * prod{(c+1)} for every alternative
        result = result.T / result.sum(axis=1)
        for i, alt_row in enumerate(result):
            df['R%d_%d' % (other_f + 1, i + 1)] = alt_row

In [22]:
def calculate_joint_probs(df, real_probs):
    sn = [slice(None)] * len(real_probs)
    for i, prob in enumerate(real_probs[0]):
        sn[0] = i
        df.loc[tuple(sn), 'JP'] = df.loc[tuple(sn),'P1'] * prob

In [23]:
def calculate_effectiveness(df, other_f_s):
    effectiveness = []
    for i, factor_s in enumerate(other_f_s):
        factor_eff = np.zeros(factor_s)
        for a in range(factor_s):
            factor_eff[a] = (df['R%d_%d' % (i + 1, a + 1)] * df['JP']).sum()
        effectiveness.append(factor_eff)
    return effectiveness

In [24]:
calculate_cross_correlation(df_outer, cross_rel, inner_f_p)
calculate_joint_probs(df_outer, outer_real_probs)
inner_effectiveness = calculate_effectiveness(df_outer, inner_f_s)

In [25]:
df_outer
# to export to excel, uncomment
#df_outer.to_excel('results.xlsx')

C           P1_           P2_           P3_  \
1 2 3 4 5 6 7 8 9 10                                                        
0 0 0 0 0 0 0 0 0 0   10.547872  3.224768e-05  2.257338e-04  3.891962e-05   
                  1    9.844680  1.108868e-05  7.762074e-05  1.338289e-05   
                  2    9.823584  3.161405e-06  2.212983e-05  3.815488e-06   
                  3    7.031914  1.357797e-05  9.504581e-05  1.638721e-05   
                  4    7.031914  1.131498e-05  7.920484e-05  1.365601e-05   
                  5   10.196276  2.461008e-05  1.722705e-04  2.970181e-05   
                  6    7.031914  2.149846e-05  1.504892e-04  2.594641e-05   
                  7    9.141489  8.825682e-06  6.177977e-05  1.065169e-05   
                  8    8.438297  1.222018e-05  8.554123e-05  1.474849e-05   
                1 0    7.735106  3.495836e-05  2.447085e-04  4.219112e-05   
                  1    7.031914  1.170854e-05  8.195979e-05  1.413100e-05   
                  2    8.930531  4.248528e-06  2.973970e-05  5.127534e-06   
                  3    7.735106  2.207896e-05  1.545527e-04  2.664703e-05   
                  4    7.031914  1.672649e-05  1.170854e-04  2.018714e-05   
                  5    7.031914  2.508973e-05  1.756281e-04  3.028071e-05   
                  6    9.141489  4.131442e-05  2.892010e-04  4.986224e-05   
                  7    2.109574  3.010768e-06  2.107537e-05  3.633685e-06   
                  8    0.703191  1.505384e-06  1.053769e-05  1.816843e-06   
                2 0    7.031914  4.019277e-05  2.813494e-04  4.850851e-05   
                  1    7.031914  1.480786e-05  1.036550e-04  1.787156e-05   
                  2    8.930531  5.373138e-06  3.761197e-05  6.484822e-06   
                  3    7.031914  2.538490e-05  1.776943e-04  3.063695e-05   
                  4    8.086701  2.432720e-05  1.702904e-04  2.936041e-05   
                  5    7.031914  3.173113e-05  2.221179e-04  3.829619e-05   
                  6    7.031914  4.019277e-05  2.813494e-04  4.850851e-05   
                  7    7.031914  1.269245e-05  8.884717e-05  1.531848e-05   
                  8    7.031914  1.903868e-05  1.332708e-04  2.297772e-05   
              1 0 0   10.547872  2.007875e-05  1.405512e-04  2.423297e-05   
                  1    9.844680  6.904271e-06  4.832990e-05  8.332741e-06   
                  2    9.823584  1.968422e-06  1.377895e-05  2.375681e-06   
...                         ...           ...           ...           ...   
1 3 3 7 2 2 6 1 2 6    0.012090  1.156553e-09  3.417088e-09  1.252932e-08   
                  7    0.016442  4.967090e-10  1.467549e-09  5.381014e-09   
                  8    0.012090  5.478408e-10  1.618621e-09  5.934942e-09   
              2 0 0    0.007673  1.546747e-10  4.569935e-10  1.675643e-09   
                  1    0.042129  3.128612e-10  9.243625e-10  3.389329e-09   
                  2    0.033101  7.023414e-11  2.075100e-10  7.608698e-10   
                  3    0.042129  5.363334e-10  1.584621e-09  5.810279e-09   
                  4    0.043032  4.565219e-10  1.348815e-09  4.945654e-09   
                  5    0.000000  0.000000e+00  0.000000e+00  0.000000e+00   
                  6    0.030092  6.065676e-10  1.792131e-09  6.571149e-09   
                  7    0.053203  3.386562e-10  1.000575e-09  3.668776e-09   
                  8    0.036110  3.447858e-10  1.018685e-09  3.735179e-09   
                1 0    0.000232  6.911850e-12  2.042138e-11  7.487838e-11   
                  1    0.001240  1.361749e-11  4.023350e-11  1.475228e-10   
                  2    0.001240  3.890712e-12  1.149529e-11  4.214938e-11   
                  3    0.001910  3.595018e-11  1.062164e-10  3.894603e-10   
                  4    0.001774  2.781859e-11  8.219130e-11  3.013681e-10   
                  5    0.000000  0.000000e+00  0.000000e+00  0.000000e+00   
                  6    0.001613  4.805030e-11  1.419668e-10  5.205449e-10   
                  7    0.0

In [26]:
df_outer[df_outer['JP'] == df_outer['JP'].max()]

,,,,,,,,,,C,P1_,P2_,P3_,P4_,P5_,P6_,P7_,P8_,P9_,...,P10,R1_1,R1_2,R1_3,R1_4,R2_1,R2_2,R2_3,R2_4,JP
1,2,3,4,5,6,7,8,9,10,,,,,,,,,,,,,,,,,,,,,
1,2,2,0,0,0,0,0,0,0,9.731419,0.000388,0.000587,0.000573,0.001484,0.000601,0.000495,0.001009,0.000476,0.001097,...,0.000723,0.299485,0.044129,0.520631,0.135754,0.420168,0.270939,0.136348,0.172546,0.000142


## Results

In [27]:
outer_real_probs

[array([ 0.3997635,  0.6002365]),
 array([ 0.05521494,  0.27784713,  0.56415954,  0.10277839]),
 array([ 0.30375674,  0.15996916,  0.49771386,  0.03856024]),
 array([ 0.25181887,  0.07402696,  0.03354394,  0.12210905,  0.19661734,
         0.16252021,  0.14958902,  0.00977461]),
 array([ 0.45844186,  0.14801249,  0.39354565]),
 array([ 0.50554618,  0.41509362,  0.0793602 ]),
 array([ 0.24403495,  0.195658  ,  0.08561875,  0.05061782,  0.1467435 ,
         0.11795038,  0.15937659]),
 array([ 0.57690604,  0.35920565,  0.06388831]),
 array([ 0.3095074 ,  0.27411999,  0.41637261]),
 array([ 0.19595404,  0.07321696,  0.02098885,  0.1240737 ,  0.11994112,
         0.14620021,  0.19808033,  0.05329754,  0.06824725])]

In [28]:
inner_effectiveness

[array([ 0.41797526,  0.04527935,  0.3928423 ,  0.14390309]),
 array([ 0.33064741,  0.32625179,  0.29859551,  0.04450529])]